Script to download structures with existing LOBSTER calculation and perform robocrys analysis on them
with standard settings, store both text output and dict output.
Requires *robocrys* environment.

In [ ]:
import logging
from mp_api.client import MPRester
import pandas as pd
from pymatgen.core import Structure
from robocrys import StructureCondenser, StructureDescriber

In [ ]:
logging.basicConfig(filename="robocrys_analysis.log",
                    filemode="w",
                    format="%(asctime)s - %(levelname)s - %(message)s",
                    level=logging.INFO)

logging.info("Started download and robocrys analysis script.")

material_ids = pd.read_csv("material_ids.csv")["mpids"].values.tolist()
all_entries = {mp_id: {} for mp_id in material_ids}

with MPRester("add-your-mp-api-key") as mpr:
    docs_summary = mpr.materials.search(material_ids=material_ids,
                                        fields=["material_id", "structure"],
                                        all_fields=False)

for doc in docs_summary:
    all_entries[str(doc.material_id)]["structure"] = doc.structure.as_dict()

In [ ]:
# Do not perform any transformations, otherwise standard robocrys settings
# Also generate two descriptions similar to Lobsterpy Analysis which_bonds argument
condenser = StructureCondenser(use_conventional_cell=False)
describer_ca = StructureDescriber(only_describe_cation_polyhedra_connectivity=True)
describer_all = StructureDescriber(only_describe_cation_polyhedra_connectivity=False)

In [ ]:
for idx, (mp_id, entry_dict) in enumerate(all_entries.items()):
    try:
        structure = Structure.from_dict(entry_dict["structure"])
    except KeyError:
        logging.info(f"No structure entry for {mp_id}, {entry_dict}.")
        all_entries[mp_id]["structure"] = None
        all_entries[mp_id]["robocrys_condensed_structure"] = None
        all_entries[mp_id]["robocrys_description_ca"] = None
        all_entries[mp_id]["robocrys_description_all"] = None
        continue

    try:
        condensed_structure = condenser.condense_structure(structure)
        all_entries[mp_id]["robocrys_condensed_structure"] = condensed_structure

        all_entries[mp_id]["robocrys_description_ca"] = describer_ca.describe(condensed_structure)
        all_entries[mp_id]["robocrys_description_all"] = describer_all.describe(condensed_structure)
    except Exception as e:
        logging.info(f"Robocrys error for entry {mp_id} ({str(e)}).")
        all_entries[mp_id]["robocrys_condensed_structure"] = None
        all_entries[mp_id]["robocrys_description_ca"] = None
        all_entries[mp_id]["robocrys_description_all"] = None
        continue
    
    if idx % 200 == 0 and idx > 0:
        df = pd.DataFrame.from_dict(all_entries, orient="index")
        df.to_json(f"df_robocrys_info_{idx}.json")

In [ ]:
df = pd.DataFrame.from_dict(all_entries, orient="index")
df.to_json("Robocrys/df_robocrys_info.json")